In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import torchvision

In [2]:
import os

# Construct the path to the 'model' folder within your project directory
model_dir = os.path.join(os.getcwd(), 'models')

# Set the TORCH_HOME environment variable to the 'model' directory path
os.environ['TORCH_HOME'] = model_dir

In [3]:
# Load the Pre-trained ResNet-50 Model
model = torchvision.models.resnet34(pretrained=True)

/home/yifeng/miniconda3/envs/attention/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/yifeng/miniconda3/envs/attention/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Remove the fully connected (FC) layer by modifying the `fc` attribute
model.fc = nn.Identity()

In [5]:
for name, _ in model.named_modules():
    print(name)


conv1
bn1
relu
maxpool
layer1
layer1.0
layer1.0.conv1
layer1.0.bn1
layer1.0.relu
layer1.0.conv2
layer1.0.bn2
layer1.1
layer1.1.conv1
layer1.1.bn1
layer1.1.relu
layer1.1.conv2
layer1.1.bn2
layer1.2
layer1.2.conv1
layer1.2.bn1
layer1.2.relu
layer1.2.conv2
layer1.2.bn2
layer2
layer2.0
layer2.0.conv1
layer2.0.bn1
layer2.0.relu
layer2.0.conv2
layer2.0.bn2
layer2.0.downsample
layer2.0.downsample.0
layer2.0.downsample.1
layer2.1
layer2.1.conv1
layer2.1.bn1
layer2.1.relu
layer2.1.conv2
layer2.1.bn2
layer2.2
layer2.2.conv1
layer2.2.bn1
layer2.2.relu
layer2.2.conv2
layer2.2.bn2
layer2.3
layer2.3.conv1
layer2.3.bn1
layer2.3.relu
layer2.3.conv2
layer2.3.bn2
layer3
layer3.0
layer3.0.conv1
layer3.0.bn1
layer3.0.relu
layer3.0.conv2
layer3.0.bn2
layer3.0.downsample
layer3.0.downsample.0
layer3.0.downsample.1
layer3.1
layer3.1.conv1
layer3.1.bn1
layer3.1.relu
layer3.1.conv2
layer3.1.bn2
layer3.2
layer3.2.conv1
layer3.2.bn1
layer3.2.relu
layer3.2.conv2
layer3.2.bn2
layer3.3
layer3.3.conv1
layer3.3.bn1


In [6]:
from torchinfo import summary

summary(model, (1, 3, 224, 224), depth=3)

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 512]                  --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│

In [7]:
import netron

torch_input = torch.randn(1, 3, 512, 512)
model = model.cuda()  # Move the model to CUDA
torch_input = torch_input.cuda()  # Move the input tensor to CUDA
# onnx_program = torch.onnx.dynamo_export(model, torch_input)
onnx_path = "./models/resnet34_pretrained.onnx"
# onnx_program.save(onnx_path)
torch.onnx.export(model, torch_input, onnx_path)
netron.start(onnx_path)

Serving './models/resnet34_pretrained.onnx' at http://localhost:8080


('localhost', 8080)

gio: http://localhost:8080: Operation not supported


In [8]:
netron.stop()

Stopping http://localhost:8080


In [9]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
class NormalizePerImage(object):
    def __call__(self, tensor):
        # Compute the mean and std for each individual image
        mean = tensor.mean([1, 2], keepdim=True)  # Compute mean along height and width
        std = tensor.std([1, 2], keepdim=True)    # Compute std along height and width
        # Normalize the image by subtracting the mean and dividing by the std
        return (tensor - mean) / (std + 1e-7)     # Adding epsilon to avoid division by zero


transform = transforms.Compose([
    transforms.ToTensor(),
    NormalizePerImage(),  # Apply the per-image normalization
])

In [11]:
import os
import shutil
from PIL import Image
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from pathlib import Path

# Define the directory path
img_path = "./img/"
output_base_path = "./img_output/"  # Base output folder for processed images

# Ensure output directory exists
if not os.path.exists(output_base_path):
    os.makedirs(output_base_path)

# Rename images sequentially in the format `0.png`, `1.png`, etc.


def rename_images_in_folder(img_folder):
    images = [f for f in os.listdir(img_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    images.sort()  # Sort images to ensure consistent renaming order

    for idx, image in enumerate(images):
        ext = os.path.splitext(image)[1]  # Get the file extension (e.g., .png, .jpg)
        new_name = f"{idx}{ext}"
        old_image_path = os.path.join(img_folder, image)
        new_image_path = os.path.join(img_folder, new_name)

        # Rename the image
        os.rename(old_image_path, new_image_path)


# Rename the images
rename_images_in_folder(img_path)

In [12]:
for idx, fname in enumerate(os.listdir(img_path)):
    if fname.endswith(('.png', '.jpg', '.jpeg')):
        img_full_path = os.path.join(img_path, fname)
        img = Image.open(img_full_path).convert('L')  # Convert to grayscale ('L')
        folder_name = os.path.join(output_base_path, str(idx))
        os.makedirs(folder_name, exist_ok=True)
        # Save the grayscale image to the output directory
        img.save(os.path.join(folder_name, f"{idx}.png"))

In [13]:
# Custom DatasetFolder to handle images without class folders
class TestImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # Gather all image file paths in the directory
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        # Open the image
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path  # No label, just return the image and its path


# Load the renamed images into a DataLoader
dataset = TestImageDataset(root_dir=img_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [14]:
selected_layer_names = [
    'layer1.0.relu', 'layer1.1.relu', 'layer1.2.relu',
    # 'layer2.0.relu', 'layer2.1.relu', 'layer2.2.relu', 'layer2.3.relu',
    # 'layer3.0.relu', 'layer3.1.relu', 'layer3.2.relu', 'layer3.3.relu', 'layer3.4.relu', 'layer3.5.relu',
    # 'layer4.0.relu', 'layer4.1.relu', 'layer4.2.relu'
]

In [15]:
def extract_feature_maps(model, selected_layer_names, input_image, relu_invocations):
    feature_maps = {}

    def hook_fn(module, input, output):
        # Find the layer's name in the model by matching the module object
        for name, mod in model.named_modules():
            if mod == module:
                # feature_maps[name] = output.detach()
                # Increase the count of ReLU invocations
                relu_invocations[name] += 1
                # print(f"Layer: {name}, ReLU invocations: {relu_invocations[name]}")

                # Store the output only for the first ReLU invocation (residual branch)
                if relu_invocations[name] == 1:
                    feature_maps[name] = output.detach()
                break

    hooks = []
    # Register hooks using the layer names
    for name, module in model.named_modules():
        # print(name)
        if name in selected_layer_names:
            # print('find')
            hook = module.register_forward_hook(hook_fn)
            hooks.append(hook)

    # Forward pass to extract feature maps
    model(input_image)

    # Remove hooks
    for hook in hooks:
        hook.remove()

    return feature_maps

In [16]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np


def apply_colormap_and_save(fmap, output_path, colormap='jet'):
    # Normalize the feature map to [0, 1] for visualization purposes
    fmap = fmap - fmap.min()  # Shift to make minimum 0
    fmap = fmap / fmap.max()  # Scale to make maximum 1

    # Convert the feature map to a numpy array
    fmap_np = fmap.cpu().numpy()

    fig, ax = plt.subplots()
    # Plot the attention map using imshow with the specified colormap
    heatmap = ax.imshow(fmap_np, cmap=colormap)

    # Turn off x-axis and y-axis
    ax.axis('off')

    # Add a color bar to the right of the plot
    cbar = plt.colorbar(heatmap, ax=ax)

    # Save the plot (including the color bar)
    plt.savefig(output_path)
    plt.close()

In [17]:
from scipy.stats import iqr, laplace


def compute_bins(fmap_np, rule='freedman-diaconis'):
    n = len(fmap_np)
    if n <= 1:
        return 10  # Default minimum number of bins for very small datasets

    data_range = fmap_np.max() - fmap_np.min()  # Range of the data

    if rule == 'sturges':
        # Sturges' rule
        return int(np.ceil(np.log2(n) + 1))

    elif rule == 'scott':
        # Scott's rule
        std = np.std(fmap_np)
        if std == 0:  # Handle the case where the standard deviation is zero
            return 10  # Default number of bins
        bin_width = 3.5 * std / (n ** (1 / 3))
        return int(np.ceil(data_range / bin_width))

    elif rule == 'freedman-diaconis':
        # Freedman-Diaconis rule
        iqr_value = iqr(fmap_np)
        if iqr_value == 0:  # Handle the case where the IQR is zero
            return 10  # Default number of bins
        bin_width = 2 * iqr_value / (n ** (1 / 3))
        if bin_width == 0:  # Ensure no division by zero
            return 10  # Default number of bins
        return int(np.ceil(data_range / bin_width))

    else:
        # Default to Sturges' rule if an unknown rule is provided
        return int(np.ceil(np.log2(n) + 1))

# Function to plot histogram with Laplace PDF and adaptive bins


def percentage_formatter(x, pos):
    return f'{x:.1f}%'


def plot_histogram_with_laplace(fmap, output_path, binning_rule='freedman-diaconis', bin_spacing=0.95):
    # Flatten the feature map to get all values in a single array
    fmap_np = fmap.cpu().numpy().flatten()

    # Compute the number of bins adaptively
    num_bins = compute_bins(fmap_np, rule=binning_rule)

    # Compute the mean and standard deviation of the feature map
    mean = np.mean(fmap_np)
    std = np.std(fmap_np)

    # Create a histogram
    fig, ax1 = plt.subplots()

    # Plot the histogram with the computed number of bins
    counts, bins, patches = ax1.hist(fmap_np, bins=num_bins, density=False, alpha=0.6, color='g', label="Channel Features", edgecolor='black', rwidth=bin_spacing)

    # Manually normalize the counts to represent percentages
    total_counts = np.sum(counts)  # Total number of counts
    percentages = (counts / total_counts) * 100  # Convert to percentage

    # Clear the plot and plot the percentages instead of counts
    ax1.cla()  # Clear the current axis
    ax1.bar(bins[:-1], percentages, width=np.diff(bins) * bin_spacing, edgecolor='black', align='edge', color='g', alpha=0.6, label="Channel Features")

    # Set labels and title for the histogram
    ax1.set_xlabel('Feature Value')
    ax1.set_ylabel('Percentage')
    # ax1.set_title(f'Histogram and Laplace PDF (mean={mean:.2f}, std={std:.2f})')

    ax1.yaxis.set_major_formatter(FuncFormatter(percentage_formatter))

    if std == 0:
        # If standard deviation is zero, set Laplace PDF to a constant (e.g., 1) to avoid division by zero
        laplace_pdf = np.ones_like(bins)  # Set PDF to a constant value (or skip this part entirely)
    else:
        # Compute the Laplace distribution using scipy's laplace function
        laplace_pdf = laplace.pdf(bins, loc=mean, scale=std / np.sqrt(2))  # Laplace scale = std / sqrt(2)

    # Create a secondary y-axis to plot the Laplace PDF
    ax2 = ax1.twinx()
    ax2.plot(bins, laplace_pdf, 'r-', label="Fitted Laplace Distribution Amplitude")
    ax2.set_ylabel('Fitted Laplace Distribution Amplitude', color='r')

    # Set limits and ticks for both axes
    # ax1.set_ylim(0, max(percentages) * 1.2)  # Adjust histogram y-axis limit for better visibility

    # # Handle the case where the Laplace PDF may contain invalid values
    # if np.all(np.isfinite(laplace_pdf)):  # Check if all values are finite (not NaN or Inf)
    #     ax2.set_ylim(0, max(laplace_pdf) * 1.2)  # Adjust Laplace y-axis limit
    # else:
    #     ax2.set_ylim(0, 1)  # Set a default y-limit if PDF contains NaN or Inf
    ax1.set_ylim(0, 8.0)
    ax2.set_ylim(0, 8)

    ax1.set_xlim(0, 1)

    ax2.tick_params(axis='y', labelcolor='r')

    # # Add legends
    # ax1.legend(loc="upper left")
    # ax2.legend(loc="upper right")
    # Combine legends from both axes
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(handles1 + handles2, labels1 + labels2, loc="upper right")  # Set combined legend position

    # Save the plot as an image
    plt.savefig(output_path)
    plt.close()

In [18]:
def compute_attention_map_and_save(fmap, output_path, colormap='jet'):
    # Flatten the feature map to get all values in a single array for computing statistics
    fmap_np = fmap.cpu().numpy().flatten()

    # Compute the mean and standard deviation of the feature map
    mean = np.mean(fmap_np)
    std = np.std(fmap_np)

    # Compute the Laplace PDF using scipy's laplace function
    laplace_pdf = laplace.pdf(fmap_np, loc=mean, scale=std / np.sqrt(2))  # Laplace scale = std / sqrt(2)

    # Convert Laplace PDF to a PyTorch tensor
    laplace_pdf_tensor = torch.tensor(laplace_pdf).to(fmap.device)

    # Compute the attention weights using torch.sigmoid(1 / (lap_pdf + 1))
    attention_weights = torch.sigmoid(1 / (laplace_pdf_tensor + 1))

    # Reshape the attention weights back to the original feature map shape
    attention_weights = attention_weights.reshape(fmap.shape).cpu().numpy()

    # # Normalize the attention weights to [0, 1] for visualization
    # attention_weights = attention_weights - attention_weights.min()
    # attention_weights = attention_weights / attention_weights.max()

    fig, ax = plt.subplots()
    ax.axis('off')
    # Plot the attention map using imshow with the specified colormap
    heatmap = ax.imshow(attention_weights, cmap=colormap, vmin=0, vmax=1)

    # Add a color bar to the right of the plot
    cbar = plt.colorbar(heatmap, ax=ax)

    # Save the plot (including the color bar)
    plt.savefig(output_path)
    plt.close()

In [19]:
device = torch.device("cuda")
model.to(device)

# Iterate over the DataLoader
for i, (input_images, _) in enumerate(dataloader):
    input_images = input_images.to(device)
    print(f"Processing batch {i+1}/{len(dataloader)}...")

    for j, input_image in enumerate(input_images):
        input_image = input_image.unsqueeze(0)  # Add batch dimension if necessary
        input_image = input_image.to(device)  # Move to the same device as the model

        # Initialize relu_invocations for the current image
        relu_invocations = {name: 0 for name in selected_layer_names}

        # Extract feature maps for the current image
        feature_maps = extract_feature_maps(model, selected_layer_names, input_image, relu_invocations)

        for layer_name, fmap in feature_maps.items():
            # Squeeze batch dimension if necessary (assuming the batch size is always 1)
            if fmap.dim() > 3:
                fmap = fmap.squeeze(0)

            C, H, W = fmap.shape
            for c in range(C):
                # Create a folder for each feature map
                fmap_folder_name = os.path.join(output_base_path, str(i), layer_name, "channel_feature_maps")
                if not os.path.exists(fmap_folder_name):
                    os.makedirs(fmap_folder_name)

                output_path_colormap = os.path.join(fmap_folder_name, f"{c}.png")
                apply_colormap_and_save(fmap[c], output_path_colormap)

                # Save the histogram with Laplace PDF
                histogram_folder_name = os.path.join(output_base_path, str(i), layer_name, "channel_histogram")
                if not os.path.exists(histogram_folder_name):
                    os.makedirs(histogram_folder_name)

                output_path_histogram = os.path.join(histogram_folder_name, f"{c}_histogram.png")
                plot_histogram_with_laplace(fmap[c], output_path_histogram)

                attention_folder_name = os.path.join(output_base_path, str(i), layer_name, "channel_attention")
                if not os.path.exists(attention_folder_name):
                    os.makedirs(attention_folder_name)

                output_path_attention = os.path.join(attention_folder_name, f"{c}_attention.png")
                compute_attention_map_and_save(fmap[c], output_path_attention)

Processing batch 1/6...


/home/yifeng/miniconda3/envs/attention/lib/python3.12/site-packages/scipy/stats/_distn_infrastructure.py:2027: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


Processing batch 2/6...
Processing batch 3/6...
Processing batch 4/6...
Processing batch 5/6...
Processing batch 6/6...
